In [7]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import seaborn as sns

In [8]:
# Load the VGG16 model as base
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))
base_model.trainable = False  # Freeze the base model

In [9]:
# Create custom model
model = models.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(2, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [10]:
# Data preparation
data_gen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_data = data_gen.flow_from_directory(
    'dataset/',
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

val_data = data_gen.flow_from_directory(
    'dataset/',
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

Found 852 images belonging to 2 classes.


Found 213 images belonging to 2 classes.


In [20]:
# Train the model
history = model.fit(train_data, validation_data=val_data, epochs=10)

Epoch 1/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 28s 1s/step - accuracy: 0.9985 - loss: 0.0102 - val_accuracy: 0.9577 - val_loss: 0.1581
Epoch 2/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 28s 1s/step - accuracy: 1.0000 - loss: 0.0078 - val_accuracy: 0.9577 - val_loss: 0.1717
Epoch 3/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 28s 1s/step - accuracy: 1.0000 - loss: 0.0055 - val_accuracy: 0.9624 - val_loss: 0.1609
Epoch 4/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 32s 1s/step - accuracy: 1.0000 - loss: 0.0049 - val_accuracy: 0.9343 - val_loss: 0.1971
Epoch 5/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 31s 1s/step - accuracy: 0.9989 - loss: 0.0079 - val_accuracy: 0.9343 - val_loss: 0.2215
Epoch 6/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 29s 1s/step - accuracy: 1.0000 - loss: 0.0059 - val_accuracy: 0.9531 - val_loss: 0.1687
Epoch 7/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 29s 1s/step - accuracy: 1.0000 - loss: 0.0033 - val_accuracy: 0.9531 - val_loss: 0.1836
Epoch 8/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 29s 1s/step - accuracy: 1.0000 - loss: 0.0025 - val_accuracy: 0.9531 - val_loss:

In [27]:
# Plot accuracy
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.savefig('accuracy_plot.png')  # Save the plot as an image
plt.show()


2025-01-08 21:01:47.527 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-08 21:01:47.528 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-08 21:01:47.542 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-08 21:01:47.738 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-08 21:01:47.739 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [23]:
# Predict validation data
val_preds = model.predict(val_data)
val_preds_classes = np.argmax(val_preds, axis=1)
val_true_classes = val_data.classes

7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 712ms/step


In [25]:
# Confusion matrix
cm = confusion_matrix(val_true_classes, val_preds_classes)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=val_data.class_indices.keys())
disp.plot(cmap=plt.cm.Blues)
plt.savefig('confusion_matrix.png')  # Save the confusion matrix as an image
plt.show()

C:\Users\Maxwell Massie\AppData\Local\Temp\ipykernel_5776\2567474120.py:6: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [19]:
# Confusion matrix
cm = confusion_matrix(val_true_classes, val_preds_classes)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=val_data.class_indices.keys())
disp.plot(cmap=plt.cm.Blues)
plt.show()

C:\Users\Maxwell Massie\AppData\Local\Temp\ipykernel_5776\382678922.py:5: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [8]:
# Save model
model.save('sawi_model.h5')